In [1]:
from bs4 import BeautifulSoup

In [2]:
import urllib3
import requests
import pandas as pd
# get source code of the page
def get_url(url):
    return urllib3.urlopen(url).read()

# makes the source tree format like 
def beautify(url):
    source = get_url(url)
    return BeautifulSoup(source,"html.parser")

In [3]:
URL = 'https://www.indeed.co.in/jobs?as_and=data+analyst&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&as_src=&salary=&radius=25&l=&fromage=any&limit=50&sort=&psf=advsrch&from=advancedsearch'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
def title(soup):
    jobs = []
    for i in soup.find_all(name="div", attrs={"class":"row"}):
        for a in i.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
              jobs.append(a["title"])
    return(jobs[:-6])

In [6]:
def comp(soup): 
    companies = []
    for i in soup.find_all(name="div", attrs={"class":"row"}):
        company = i.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for i in company:
                companies.append(i.text.strip())
        else:
            b = i.find_all(name="span", attrs={"class":"result-link-source"})
            for j in b:
                companies.append(j.text.strip())
    return(companies[:-6])

In [7]:
def loc(soup): 
    locations = []
    spans = soup.findAll('span', attrs={'class': 'location'})
    for i in spans:
        locations.append(i.text)
    return(locations)

In [8]:
def sal(soup): 
    salaries = []
    for i in soup.find_all(name='div', attrs={'class':'row'}):
        sal = i.find(name='div', attrs={'class':'salarySnippet'})
        salaries.append(sal.text.strip() if sal else 'Nil')
    return(salaries[:-6])

In [9]:
jobs = title(soup)
companies=comp(soup)
locations=loc(soup)
salary=sal(soup)

In [10]:
job_list=pd.DataFrame({'Job Title':jobs, 'Company':companies,'Location':locations,'Salary':salary})

In [11]:
job_list=job_list.to_excel("job_list.xls")